In [1]:
# Install Libraries
!pip install textblob
!pip install tweepy
!pip install pycountry
!pip install langdetect

In [2]:
# Import Libraries

from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('vader_lexicon')
import pycountry
import re
import string

from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nehapatil/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Authentication
consumerKey = 'JvaUYSYyckryhmmCNQLp4DuK6'
consumerSecret = '3p07FNUAKUJMpg9ANpCgEBrVAqw8H4dcAVACjWO1FZEDBsNsg9'
accessToken = '1276677741799186432-bxjvVd7dX4wk4P7cegysfHi1fYFbYu'
accessTokenSecret = 'XAAKBrOQMw0CavFPTeWptsv8un1xrVNhVgDc1dBKCBqWB'

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
#Sentiment Analysis

def percentage(part,whole):
    return 100 * float(part)/float(whole) 

keyword = input("Please enter keyword or hashtag to search: ")
noOfTweet = int(input ("Please enter how many tweets to analyze: "))


tweets = tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
positive  = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []
dates = []

for tweet in tweets:
    
    #print(tweet.text)
    dates.append(tweet.created_at)
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
    
    if neg > pos:
        negative_list.append(tweet.text)
        negative += 1

    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1
    
    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')
print(dates)


Please enter keyword or hashtag to search: crude oil
Please enter how many tweets to analyze: 10000


In [ ]:
#Cleaning Text (RT, Punctuation etc)
#Creating new dataframe and new features
tw_list = pd.DataFrame(tweet_list)
tw_list['Dates'] = dates
tw_list.drop_duplicates(inplace = True)
tw_list["text"] = tw_list[0]
#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()

tw_list.head(10)

In [ ]:
#Calculating Negative, Positive, Neutral and Compound values
tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].iteritems():
 score = SentimentIntensityAnalyzer().polarity_scores(row)
 neg = score['neg']
 neu = score['neu']
 pos = score['pos']
 comp = score['compound']
 if neg > pos:
    tw_list.loc[index, 'sentiment'] = "negative"
 elif pos > neg:
    tw_list.loc[index, 'sentiment'] = "positive"
 else:
    tw_list.loc[index, 'sentiment'] = "neutral"
    tw_list.loc[index, 'neg'] = neg
    tw_list.loc[index, 'neu'] = neu
    tw_list.loc[index, 'pos'] = pos
    tw_list.loc[index, 'compound'] = comp
    
tw_list.tail(10)

In [ ]:
#Creating new data frames for all sentiments (positive, negative and neutral)
tw_list_negative = tw_list[tw_list["sentiment"]=="negative"]
tw_list_positive = tw_list[tw_list["sentiment"]=="positive"]
tw_list_neutral = tw_list[tw_list["sentiment"]=="neutral"]



In [ ]:
def count_values_in_column(data,feature):
 total=data.loc[:,feature].value_counts(dropna=False)
 percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
 return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])
#Count_values for sentiment
count_values_in_column(tw_list,"sentiment")

In [ ]:
tw_list.tail()
tw_list.to_csv('tweets_on_crude_oil.csv')

In [ ]:
tw_list.drop(["text","subjectivity","sentiment"], axis=1, inplace=True)
tw_list.tail()

In [ ]:
data = pd.read_csv("tweets_on_crude_oil.csv")
data.tail()

In [ ]:
data.drop(["0", "Unnamed: 0", "text","subjectivity","sentiment", "neg", "neu", "pos", "compound"],axis=1,inplace=True)

In [ ]:
data.head()
dates = data["Dates"].to_list()
print(len(data))
import pandas as pd
date1 = '1972-08-13'
date2 = '2021-04-20'
mydates = pd.date_range(date1, date2).tolist()

data.index = mydates

In [ ]:
data.index = pd.DatetimeIndex(data.index)
data = data[~data.index.duplicated(keep='first')]

In [ ]:
idx = pd.date_range(min(data.index),max(data.index))

data = data.reindex(idx, fill_value=0)

print(data)

import matplotlib.pyplot as plt
plt.plot(dates,data["polarity"])

plt.gcf().set_size_inches(20, 10)

plt.ylabel("Polarity", fontsize=18)
plt.xlabel("Date", fontsize=18)
plt.title("Sentiment of News Articles", fontsize=18)

plt.show()

In [ ]:
data.tail()
#dat.to_csv('sa_news.csv')